In [9]:
import os,nbloader,warnings,sys,time
warnings.filterwarnings("ignore")

import numpy as np
import pickle

from multigoal_imitation_learning import train_imitation_learning

import itertools
from multiprocessing import Process

if __name__ == "__main__":
    print ("Python version is [%s]"%(sys.version_info[0]))

Python version is [2]


In [10]:
class grid_maker(object):
    def __init__(self,*_arg):
        self.arg = _arg
        self.nArg = len(self.arg) # Number of total lists
        _product = itertools.product(*self.arg); _nIter = 0
        for x in _product: _nIter += 1
        self.nIter = _nIter
        self.paramList = ['']*self.nIter
        self.idxList = ['']*self.nIter
        _product = itertools.product(*self.arg);
        for idx,x in enumerate(_product):
            self.paramList[idx] = x

In [11]:
def get_properIdx(_processID,_maxProcessID,_nTask):
    ret = []
    if _processID > _nTask: return ret
    if _processID > _maxProcessID: return ret
    m = (_nTask-_processID-1) // _maxProcessID
    for i in range(m+1):
        ret.append(i*_maxProcessID+_processID)
    return ret
if __name__ == "__main__":
    maxProcessID,nTask = 8,11
    print ("\nmaxProcessID:[%d], nTask:[%d]"%(maxProcessID,nTask ))
    for processID in range(maxProcessID):
        ids = get_properIdx(_processID=processID,_maxProcessID=maxProcessID,_nTask=nTask)
        print (" processID:[%d] %s"%(processID,ids))


maxProcessID:[8], nTask:[11]
 processID:[0] [0, 8]
 processID:[1] [1, 9]
 processID:[2] [2, 10]
 processID:[3] [3]
 processID:[4] [4]
 processID:[5] [5]
 processID:[6] [6]
 processID:[7] [7]


In [12]:
kl_targ_list=[3e-3, 3e-2, 3e-4]
entcoeff_list=[1e-3, 1e-5, 1e-7]
mdn_weigth_list=["sparsemax","softmax"]
n_mixture_list=[4,2,6,8]
epi_size_list=[100,200,300,400,500]
seed_list=[1,2,3,4,5]

def get_multigoal_config(_processID=0,_maxProcessID=2,_maxGPU=1,_DO_SHUFFLE=False):
    # Get total configurations
    _G = grid_maker(kl_targ_list,seed_list,entcoeff_list,n_mixture_list,epi_size_list,mdn_weigth_list)
    # Get current configurations
    _ids = get_properIdx(_processID,_maxProcessID,_nTask=_G.nIter)
    _paramsList = list(_G.paramList[i] for i in _ids) 
    # Set GPU ID
    _GPU_ID = (_processID % _maxGPU)
    # Suffle if required
    if _DO_SHUFFLE:
        shuffle(_paramsList)
    return _paramsList,_GPU_ID

if __name__ == "__main__":
    processID = 5
    maxProcessID = 32
    maxGPU = 4
    paramsList,GPU_ID = get_multigoal_config(processID,maxProcessID,maxGPU)
    print ("GPU_ID:[%d]"%(GPU_ID))
    for pIdx,params in enumerate(paramsList): # For all current configurations
        print (pIdx,params) 

GPU_ID:[1]
(0, (0.003, 1, 0.001, 4, 300, 'softmax'))
(1, (0.003, 1, 0.001, 8, 400, 'softmax'))
(2, (0.003, 1, 1e-05, 6, 500, 'softmax'))
(3, (0.003, 1, 1e-07, 6, 100, 'softmax'))
(4, (0.003, 2, 0.001, 2, 200, 'softmax'))
(5, (0.003, 2, 1e-05, 4, 300, 'softmax'))
(6, (0.003, 2, 1e-05, 8, 400, 'softmax'))
(7, (0.003, 2, 1e-07, 6, 500, 'softmax'))
(8, (0.003, 3, 0.001, 6, 100, 'softmax'))
(9, (0.003, 3, 1e-05, 2, 200, 'softmax'))
(10, (0.003, 3, 1e-07, 4, 300, 'softmax'))
(11, (0.003, 3, 1e-07, 8, 400, 'softmax'))
(12, (0.003, 4, 0.001, 6, 500, 'softmax'))
(13, (0.003, 4, 1e-05, 6, 100, 'softmax'))
(14, (0.003, 4, 1e-07, 2, 200, 'softmax'))
(15, (0.003, 5, 0.001, 4, 300, 'softmax'))
(16, (0.003, 5, 0.001, 8, 400, 'softmax'))
(17, (0.003, 5, 1e-05, 6, 500, 'softmax'))
(18, (0.003, 5, 1e-07, 6, 100, 'softmax'))
(19, (0.03, 1, 0.001, 2, 200, 'softmax'))
(20, (0.03, 1, 1e-05, 4, 300, 'softmax'))
(21, (0.03, 1, 1e-05, 8, 400, 'softmax'))
(22, (0.03, 1, 1e-07, 6, 500, 'softmax'))
(23, (0.03, 2,

In [13]:
def train_imitation_learning_multiprocessing(parameter_list=None,GPU_ID=0,verbose=False):
    for pIdx,params in enumerate(parameter_list): # For all current configurations
        kl_targ,seed,entcoeff,n_mixture,demo_size,mdn_weight=params
        train_imitation_learning(seed,kl_targ,entcoeff,n_mixture,demo_size,mdn_weight,GPU_ID=GPU_ID,verbose=verbose)

if __name__ == "__main__":
    processID = 0
    maxProcessID = 1792
    parameter_list,GPU_ID = get_multigoal_config(processID,maxProcessID)
#     train_imitation_learning_multiprocessing(parameter_list=parameter_list,GPU_ID=GPU_ID,verbose=True)

In [14]:
class worker_class(Process):
    def __init__(self,_idx=0,_maxProcessID=8,_maxGPU=8,_name='worker',_FLAG='',_period=1,_maxTick=5,_VERBOSE=True):
        Process.__init__(self)
        # Initialize class
        self.idx = _idx
        self.maxProcessID = _maxProcessID
        self.maxGPU = _maxGPU
        self.name = _name
        self.FLAG = _FLAG # Running flag 
        self.VERBOSE = _VERBOSE
        # Initialize Process 
        self.setName = 'T_'+self.name 
        self.args = (self.FLAG,)
        self.daemon = True # Make sure that each child is killed when the parent is dead. (?!)
        # Print
        if self.VERBOSE:
            print ("[%s] Instantiated."%(self.name))
    def run(self):
        print ("Starting [%s]"%(self.name))
        time.sleep(1e-1)
        # Get configurations
        parameter_list,GPU_ID = get_multigoal_config(self.idx,self.maxProcessID,self.maxGPU)
        print ("processID:[%d/%d] GPU_ID:[%d] #Config:[%d]"
               %(self.idx,self.maxProcessID,GPU_ID,len(parameter_list)))
        #Run
        train_imitation_learning_multiprocessing(parameter_list=parameter_list,GPU_ID=GPU_ID,verbose=True)
        print ("[%s] Done."%(self.name))
        
if __name__ == "__main__":
    print ("worker_class defined.")

worker_class defined.
